<a href="https://colab.research.google.com/github/hwanys2/AI-math/blob/main/5.%ED%9A%8C%EA%B7%80%EC%99%80%EB%B6%84%EB%A5%98/5_2.k_means_%EA%B5%B0%EC%A7%91%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 군집화
정답을 미리 알고 있지 않는 비지도학습의 대표적인 예로 군집화에 대해 살펴보겠습니다.  
데이터는 kaggle에서 제공하는 데이터를 사용하였습니다.  
[KAGGLE](https://www.kaggle.com)

다음 책에 나온 내용을 참조하여 최신데이터로 수정 및 추가적인 내용을 다루었습니다.  
[수학과 함께하는 AI기초](http://www.yes24.com/Product/Goods/92450319)

많은 데이터를 다룰 때 pandas라는 라이브러리가 유용합니다.  
다음과 같이 import 하겠습니다.

In [ ]:
import pandas as pd

Kaggle에서 받은 자료를 제가 웹에 올려놓고 바로 사용할 수 있게 해 놓았습니다.  
다음과 같이 데이터를 불러올 수 있습니다.  
pandas의 데이터프레임 형식으로 불러오는데 우선 엑셀과 비슷하다고 생각하시면 될 것 같습니다.

In [ ]:
data_url= 'https://raw.githubusercontent.com/hwanys2/data/main/players_21.csv'
fifa2021 = pd.read_csv(data_url)
fifa2021

이 자료는 18943명의 선수들의 106 종류의 데이터를 담고 있습니다.  
간단히 살펴보시면 이름, 국적, 나이, 선수별 각종 특성, 연봉 등 다양한 정보가 있음을 확인할 수 있습니다.  
이 중 우리나라 선수만 모아서 살펴보도록 하겠습니다.  
다음과 같이 nationality가 Korea Republic 인 자료만 korea 라는 데이터프레임으로 새로 만들겠습니다.  

In [ ]:
# 우리나라 선수만 모아보기
korea = fifa2021[fifa2021['nationality'] == "Korea Republic"]
korea



---



---



### 실습
나이가 35살보다 많은 선수들만 upper_age_35 라는 데이터프레임으로 만들어 보세요.  
upper_age_35 = 로 시작해야겠죠?



---



---



# 한국 선수 분석

한국선수의 'short_name','shooting','passing','dribbling','defending' 만을 모아서 시각적으로 표현해보겠습니다.  
이렇게 모오는 방법은 loc라는 명령어가 존재합니다.  
사용방법은 다음과 같습니다.


```
korea.loc[행의 범위, 열의범위]
```
우리는 모든 선수들을 대상으로 할 것이기에 행은 전체를 나타내고, 열은 위의 특성들을 리스트에 담아 적으면 됩니다.  다음과 같습니다.


```
korea.loc[:,['short_name','shooting','passing','dribbling','defending']]
```




dropna()는 빈칸을 제거하는 명령입니다.  
이러한 정보만 담긴 데이터프레임을 korea_features 라고 하겠습니다.

In [ ]:
korea_features = korea.loc[:,['short_name','shooting','passing','dribbling','defending']].dropna()
korea_features

이 자료를 산점도를 이용하여 표현해 보겠습니다.  
슈팅과 패싱 능력치를 산점도로 표현하겠습니다.

In [ ]:
import matplotlib.pyplot as plt
plt.title("shooting & passing")      # 그래프의 제목 입력 
plt.xlabel('shooting')                 # x축 이름 입력 
plt.ylabel('passing')               # y축 이름 입력 
plt.scatter(korea_features['shooting'],korea_features['passing'])     # 산점도 그리기

### 실습
dribbling과 defending을 산점도로 나타내 보세요.



---



---



이제 슈팅과 패싱능력만을 가지고 군집화 해보도록 하겠습니다.  
먼저 슈팅과 패싱능력만을 포함한 자료를 group_korea 라는 데이터프레임으로 만들겠습니다.

In [ ]:
group_korea = korea.loc[:,['shooting','passing']].dropna()
group_korea

# K-means 군집화란?

비계층적 군집화 방법으로 전체 데이터를 상대적으로 유사한 K개의 군집으로 구분하는 방법입니다.

![이미지](https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F09e0386f-0205-47ad-ac8d-0a90f55690d7%2FUnknown.png?table=block&id=ae88daf5-72dd-4adb-a76e-8cdeb51e9499&spaceId=0f859e6e-7f7d-4665-851d-3e7244ab307b&width=2000&userId=dfd283da-c918-4fc5-8a30-95ceb1b2371a&cache=v2)

K-means 군집화의 과정은 클러스터의 개수를 결정하게 되면 데이터 공간에 K개의 클러스터가 형성되며 각 클러스터의 중심은 해당 클러스터에 포함된 데이터의 평균으로 조정하게 됩니다. 클러스터의 중심이 변하지 않을 때까지 클러스터 중심을 조정하며 군집을 생성합니다.
<br>
<aside>
💡 1. 클러스터 개수 K값 결정하기 <br>
2. 데이터 공간에 클러스터 중심 K개 할당하기<br>
3. 각 클러스터 중심을 해당 클러스터 데이터의 평균으로 재조정하기<br>
4. 클러스터 중심이 변하지 않을 때까지 3.을 반복 시행
<br>
</aside>

K-means 군집화는 빠르고 간단하게 군집화가 가능하다는 강점이 있고 적절하지 못한 K를 설정할 경우 그 결과가 좋지 못할 수 있다는 약점이 있습니다.

데이터프레임의 자료를 사용하기 쉽게 벡터형태로 바꾸겠습니다.  
정확히는 array로 바꾸겠습니다. numpy 라이브러리를 사용합니다.

In [ ]:
import numpy as np
X = np.array(group_korea)    # 자료를 벡터로 변환

군집화 하기위한 클러스터 개수를 지정하겠습니다.  
3개의 군집으로 나눠보는 실습을 하겠습니다.  
먼저 필요한 것은 임의의 군집 센터들을 잡는 것입니다.  
랜덤하게 3개의 센터를 만들겠습니다.  
이 출력 결과는 모두가 다르게 나옵니다.

In [ ]:
k = 3                                # 군집화 하려는 개수 즉 3그룹으로 나누려고 함.
C_x = np.random.choice(X[:, 0],k)    # 슈팅 점수중 k개 추출
C_y = np.random.choice(X[:, 1],k)    # 패싱 점수중 k개 추출
C = np.array(list(zip(C_x, C_y)))
print(C)

초기의 센터 위치들을 그래프로 확인해보겠습니다.

In [ ]:
# 그래프로 그리기. 중심은 별로 표시
plt.title("shooting & passing")    # 타이틀
plt.xlabel('shooting')
plt.ylabel('passing')
plt.scatter(group_korea['shooting'],group_korea['passing'], marker = '^', c = 'blue', s = 10, label = 'players')
plt.scatter(C_x, C_y, marker ='*', s=200, c= 'black', label='centroids')
plt.legend(loc = 'best')    # 범례를 좋은위치에

이제 이 별들의 위치를 계속 바꿔가면서 더이상 변화하지 않을 때까지 센터의 위치를 바꾸는 작업을 진행하겠습니다.  

그러기 위해 각 선수들의 점들과 센터들과의 거리를 구하기 위해 거리구하는 함수를 정의하겠습니다.

In [ ]:
# 유클리디안 거리 계산 함수 만들기
def Distance(A, B):
    return np.sqrt(np.sum(np.power((A-B),2)))

또 계산에 앞서 clusters를 만들겠습니다. clusters에는 각 선수들이 3개의 별 중 어느 별에 가장 가까운지에 대한 정보를 담을 것입니다.  
각 별은 0, 1, 2로 표현하여 만약 clusters = [0, 0, 1, 2, 0, .....] 과 같이 표현된다면  
맨 처음 선수와 두번 째선수는 0번 별에 가장 가까운 것이고, 세 번째 선수는 1번별, 네번째 선수는 2번별에 가깝다는 정보를 담을 것입니다.  

In [ ]:
clusters = np.zeros(len(X))     # 데이터의 개수 만큼 0

그리고 이를 바탕으로 clusters가 같은 값을 가진 선수들의 x, y좌표의 평균을 계산하여 그 값을 새로운 센터로 지정하게 됩니다.  
이렇게 지정된 새로운 C와 이전의 C_old와의 거리를 구해 이 거리가 0이 될 떄까지 반복수행하게 됩니다.  
이 거리를 flag 라고 하겠습니다.  
임의로 선택된 센터(C)와 C_old는 모두 원점이라 가정하면 다음과 같은 결과가 나옵니다.  
이 flag 값은 실행하는 사람의 센터 값이 다르기 때문에 모두 다르게 나옵니다.

In [ ]:
C_old = np.zeros(C.shape)       # C 와 같은 모양의 0행렬
flag = Distance(C, C_old)       # 새로운 중심과 이전 중심의 거리를 구함. 
flag

이제 군집화를 실제 진행해보도록 하겠습니다.  
코드가 다소 길어 주석을 살펴보시기 바랍니다.

In [ ]:
from copy import deepcopy

distances = []

while flag != 0:                         # 새로 구해진 중심과 직전의 중심과 같아질 떄까지 반복하라

    for i in range(len(X)):              # 한국선수의 수만큼 반복, 모든 선수를 중심과 가장 가까운 곳을 찾는 시도
        for j in range(k):               # 각 선수마다 k개의 중심점 중 거리를 구해서 가장 가까운 곳을 찾는 시도
            temp = Distance(X[i], C[j])  # 선수와 k개의 중심점과 거리를 temp에 저장
            distances.append(temp)       # [10, 20, 41] 이런식으로 각 중심과의 거리가 담기게 됨. 이때 가장 작은 것이 필요.
    
        cluster = np.argmin(distances)   # 거리가 가장 작은 것의 인덱스를 cluster로 지정. 즉 위의 [10, 20, 41]이라면 0번째를 cluster에 저장. np.argmin이 최소값의 인덱스출력
        clusters[i] = cluster            # 모든 선수들이 어떤 중심점에 가까운지 분류됨.
        distances = []                   # 다음 선수의 거리를 측정하기 전에 distance 초기화
  
    C_old = deepcopy(C)                  # 딥카피는 후속 작업에 영향을 안받아서 사용해야함. 

    for i in range(k):                   # 새로운 중심점을 분류된 집합의 평균 x,y 좌표로 지정함.
        points = [X[j] for j in range(len(X)) if clusters[j]==i]
        C[i] = np.mean(points, axis=0)
  

    flag = Distance(C, C_old)            # 위 과정을 반복하면서 새로운 중심점과 이전의 중심점의 거리를 구함. 이게 0이 될때까지 지속함.
print(C)

군집화된 위치별로 다른 색을 이용하여 그림으로 표현해보겠습니다.  
다음과 같습니다.

In [ ]:
plt.scatter(X[clusters == 0, 0], X[clusters == 0, 1], s=50, c='red', marker ='o', edgecolor = 'black', label = 'Middle')
plt.scatter(X[clusters == 1, 0], X[clusters == 1, 1], s=50, c='yellow', marker ='x', edgecolor = 'black', label = 'High')
plt.scatter(X[clusters == 2, 0], X[clusters == 2, 1], s=50, c='blue', marker ='^', edgecolor = 'black', label = 'Low')

plt.scatter(C[:,0], C[:,1], s = 250, marker = '*', c = 'white', edgecolor='black',label='Centroids')
plt.legend()
plt.grid()

### 실습

K-means 군집화란?  이라고 표시된 부분으로 넘어가서  
무엇을 수정해야 4개의 군집화가 될지 코드를 수정하며 실행하여 4개로 군집화를 진행해보세요.



---



---



---



---



K-means 군집화의 원리를 바탕으로 진행해보았습니다.  
이제 사이킷런에서 이미 제공하고 있는 모델을 사용하여 군집활를 진행해보도록 하겠습니다.  
우선 다음과 같이 불러 옵니다.

In [ ]:
import pandas as pd 
from sklearn.cluster import KMeans # model 

방법은 선형회귀와 비슷합니다.  
객체를 생성하고 자료를 fit 해주면 됩니다.  
클러스터의 개수만 지정해주면 자동으로 군집화를 완료해줍니다.  
여기서는 데이터의 형태가 데이터프레임을 그대로 넣어줘도 가능합니다.  


In [ ]:
model = KMeans(n_clusters=6)           ### 클러스터 개수 지정 숫자 변경하면 됨.
dataDF = group_korea[['shooting', 'passing']]
model.fit(dataDF) 

위에서 만들었던 clusters 값을 predict를 통해 확인할 수 있습니다.  
이를 바탕으로 그래프로 표현하면 다음과 같습니다.

In [ ]:
clusters = model.predict(dataDF) 
centers = model.cluster_centers_ 
plt.scatter(x=dataDF['shooting'], y=dataDF['passing'], c=clusters) 
plt.scatter(x=centers[:,0], y=centers[:,1], marker='*', s=250, c = 'white', edgecolor='black',)

사이킷런을 활용하여 군집화 한 과정을 모아 보겠습니다.  
데이터 수집 및 전처리 부터 결과까지 아래의 짧은 코드로 프로그래밍이 가능해집니다.  
위에서 했던 과정과 비교해 보시길 바랍니다.

In [ ]:
import pandas as pd 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

# 데이터 불러오기
data_url= 'https://raw.githubusercontent.com/hwanys2/data/main/players_21.csv'
fifa2021 = pd.read_csv(data_url)

# 데이터 전처리(사용하고 싶은 부분만 모으기)
korea = fifa2021[fifa2021['nationality'] == "Korea Republic"]
group_korea = korea.loc[:,['shooting','passing']].dropna()

# 사이킷런에 모델에 적용시킨다.
model = KMeans(n_clusters=3)    # n_clusters 값이 군집의 갯수이다.
model.fit(group_korea)

# 그래프 표현
clusters = model.predict(group_korea) 
centers = model.cluster_centers_ 
plt.scatter(x=group_korea['shooting'], y=group_korea['passing'], c=clusters) 
plt.scatter(x=centers[:,0], y=centers[:,1], marker='*', s=250, c = 'white', edgecolor='black',)